In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz

# 1. Binance API

#### <b> Columns meanings: </b>
1) <b>Open time:</b> The timestamp marking the start of the kline/candlestick interval (in milliseconds).
2) <b>Open:</b> The price of the first trade during the kline/candlestick interval.
3) <b>High:</b> The highest price reached during the kline/candlestick interval.
4) <b>Low:</b> The lowest price reached during the kline/candlestick interval.
5) <b>Close:</b> The price of the last trade during the kline/candlestick interval.
6) <b>Volume:</b> The total trading volume of the base asset during the kline/candlestick interval. (<b>Example:</b> If the trading pair is BTC/USDT and the volume is 100, it means 100 BTC were traded during the interval.)
7) <b>Close time:</b> The timestamp marking the end of the kline/candlestick interval (in milliseconds).
8) <b>Quote asset volume:</b> The total trading volume of the quote asset during the kline/candlestick interval. (<b>Example:</b>  For the BTC/USDT pair, if the quote asset volume is 500,000, it means the equivalent of 500,000 USDT was traded during the interval.)
9) <b>Number of trades:</b> The total number of trades executed during the kline/candlestick interval. (<b>Example:</b>  If the number of trades is 250, it means there were 250 separate transactions during the interval.)
10) <b>Taker buy base asset volume:</b> The volume of the base asset bought by takers during the kline/candlestick interval. (<b>Example:</b>  For BTC/USDT, if the taker buy base asset volume is 60, it means takers bought 60 BTC during the interval.
<b>Takers:</b> Takers are traders who execute market orders that match existing orders on the order book, essentially removing liquidity.)
11) <b>Taker buy quote asset volume:</b> The volume of the quote asset spent to buy the base asset by takers during the kline/candlestick interval. (<b>Example:</b> For BTC/USDT, if the taker buy quote asset volume is 300,000, it means takers spent 300,000 USDT to buy BTC during the interval.)
12) <b>Ignore:</b> A placeholder field that can be ignored. It does not hold meaningful data.

In [2]:
def get_historical_klines(symbol, interval, start_str, end_str= None):
    """
        params: 
            'symbol' - currency, e.g. 'BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'ADAUSDT', 'XRPUSDT', 'SOLUSDT'
            'interval' - one of these values '1m', '3m', '5m', '15m', '30m', '1h', '2h',
                                            '4h', '6h', '8h', '12h', '1d', '3d', '1w', '1M'.
    """
    url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_str,
    }
    if end_str:
        params['endTime'] = end_str
    response = requests.get(url, params=params)
    data = response.json()
    print(type(data))
    
    # Parse the data into a DataFrame
    columns = [
        'Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
        'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
        'Taker buy quote asset volume', 'Ignore'
    ]
    df = pd.DataFrame(data, columns=columns)
    
    # Convert the timestamp columns to datetime
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
    df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
    
    # Set the index to the open time
    #df.set_index('Open time', inplace=True)
    df.rename(
        columns={
                "Open time": "Open_time", 
                "Close time": "Close_time",
                "Quote asset volume": "Quote_asset_volume",
                "Number of trades": "Number_of_trades",
                "Taker buy base asset volume": "Taker_buy_base_asset_volume",
                "Taker buy quote asset volume": "Taker_buy_quote_asset_volume"
        }
    )
    
    return df
    

In [3]:
# Define interaval

# Define startTime
date_string = datetime(
    year=2015, month=9, day=24, 
    hour=0, minute=0, second=0,
    tzinfo=pytz.UTC
)
start_str = int(date_string.timestamp() * 1000)

# Define endTime
end_date_string = datetime(
    year=2019, month=10, day=24, 
    hour=23, minute=59, second=59,
    tzinfo=pytz.UTC
)
end_start_str = int(end_date_string.timestamp() * 1000)


In [5]:
# Example usage

# Define currency
symbol = 'BTCUSDT'

# Define interval
interval = '1h' 

btc_data = get_historical_klines(symbol, interval, start_str, end_start_str) # end_start_str
print(f"Data for interval: {btc_data.index.min()}-{btc_data.index.max()}")

btc_data.head()

<class 'list'>
Data for interval: 0-499


,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2017-08-17 04:00:00,4261.48000000,4313.62000000,4261.32000000,4308.83000000,47.18100900,2017-08-17 04:59:59.999,202366.13839304,171,35.16050300,150952.47794304,0
1,2017-08-17 05:00:00,4308.83000000,4328.69000000,4291.37000000,4315.32000000,23.23491600,2017-08-17 05:59:59.999,100304.82356749,102,21.44807100,92608.27972836,0
2,2017-08-17 06:00:00,4330.29000000,4345.45000000,4309.37000000,4324.35000000,7.22969100,2017-08-17 06:59:59.999,31282.31266989,36,4.80286100,20795.31722405,0
3,2017-08-17 07:00:00,4316.62000000,4349.99000000,4287.41000000,4349.99000000,4.44324900,2017-08-17 07:59:59.999,19241.05829986,25,2.60229200,11291.34701534,0
4,2017-08-17 08:00:00,4333.32000000,4377.85000000,4333.32000000,4360.69000000,0.97280700,2017-08-17 08:59:59.999,4239.50358563,28,0.81465500,3552.74681715,0


# =============================================================================
# Далее по мере внедрения новых источников данных надо организовать вкладки для них 
# =============================================================================

# 2.Alpha Vantage API for abroad stocks!!! WHY here isn't today data but only yesterday !!!

### https://www.alphavantage.co/documentation/

# !!! WHY here isn't today data but only yesterday !!! возможно потому что биржа открыта с 16:30 по 23:00 по мск 

In [197]:
def get_stock_data(symbol, api_key):
    """

    """
    api_key = 'QIQY5CY2F1DQ6CA8' # !!! HIDE 
    url = f'https://www.alphavantage.co/query'
    
    params = {
        'function': 'TIME_SERIES_INTRADAY', # TIME_SERIES_INTRADAY, TIME_SERIES_DAILY, TIME_SERIES_WEEKLY, TIME_SERIES_WEEKLY (+'_ADJUSTED')
        'interval': '1min',
        'symbol': symbol,
        'apikey': api_key,
        'outputsize': 'full',
        #'month':'2024-06' # to query a specific month in history
    }
    
    response = requests.get(url, params=params).json()
    #res_dataframe = pd.DataFrame(pd.DataFrame(response[list(response.keys())[-1]]).transpose())
    #res_dataframe['Symbol'] = response[list(response.keys())[0]]['2. Symbol']
    #res_dataframe['LastRefreshed'] = response[list(response.keys())[0]]['3. Last Refreshed']
    #res_dataframe['Interval'] = response[list(response.keys())[0]]['4. Interval']
    #res_dataframe['OutputSize'] = response[list(response.keys())[0]]['5. Output Size']
    #res_dataframe['Time Zone'] = response[list(response.keys())[0]]['6. Time Zone']
    return response


response_stock_data = get_stock_data('AAPL', api_key) # AAPL, COP , GAZP.ME

# Print a sample of the data
response_stock_data

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [177]:
def get_technical_indicators(symbol, interval, time_period, series_type, api_key):
    api_key = 'QIQY5CY2F1DQ6CA8' # !!! HIDE Alpha Vantage API key
    url = 'https://www.alphavantage.co/query'
    
    params = {
        'function': 'SMA', # SMA(Simple Moving Average), EMA(Exponential Moving Average) , WMA(Weighted Moving Average), DEMA and many many other 
        'symbol': symbol,
        'interval': interval,
        'time_period': time_period,
        'series_type': series_type,
        'apikey': api_key
    }
    response = requests.get(url, params=params).json()
    res_dataframe = pd.DataFrame(pd.DataFrame(response[list(response.keys())[-1]]).transpose())
    res_dataframe['Symbol'] = response[list(response.keys())[0]]['1: Symbol']
    res_dataframe['Indicator'] = response[list(response.keys())[0]]['2: Indicator']
    res_dataframe['LastRefreshed'] = response[list(response.keys())[0]]['3: Last Refreshed']
    res_dataframe['Interval'] = response[list(response.keys())[0]]['4: Interval']
    res_dataframe['TimePeriod'] = response[list(response.keys())[0]]['5: Time Period']
    res_dataframe['SeriesType'] = response[list(response.keys())[0]]['6: Series Type']
    res_dataframe['TimeZone'] = response[list(response.keys())[0]]['7: Time Zone']

    return res_dataframe


# Parameters for the SMA
symbol = 'AAPL'  # Example stock symbol
interval = 'daily'  # Interval can be '1min', '5min', '15min', '30min', '60min', 'daily', 'weekly', 'monthly'
time_period = 20  # The number of data points used to calculate the SMA(Simple Moving Average) 
series_type = 'close'  # The desired price type in the time series ('close', 'open', 'high', 'low')

# Fetch SMA data
sma_data = get_technical_indicators(symbol, interval, time_period, series_type, api_key)

# Print a sample of the data
sma_data

,SMA,Symbol,Indicator,LastRefreshed,Interval,TimePeriod,SeriesType,TimeZone
2024-06-25,202.2425,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
2024-06-24,201.2880,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
2024-06-21,200.2250,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
2024-06-20,199.3955,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
2024-06-18,198.5290,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
...,...,...,...,...,...,...,...,...
1999-12-03,0.7153,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
1999-12-02,0.7035,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
1999-12-01,0.6927,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern
1999-11-30,0.6840,AAPL,Simple Moving Average (SMA),2024-06-25,daily,20,close,US/Eastern


#  3. yfinance - extra example 

In [9]:
import yfinance as yf

# Define the ticker symbol
symbol = 'AAPL'

# Define the date range (1 year ago from today)
end_date = datetime.today() # datetime(2024, 6, 26, 9, 30, 0) #datetime.today()
start_date = end_date - timedelta(days=6)

# Fetch historical data
data = yf.download(symbol, start=start_date, end=end_date, interval='1m')

# Print the data
print(data)

[*********************100%%**********************]  1 of 1 completed

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-07-02 14:29:00-04:00  220.029999  220.070007  220.000000  220.029999   
2024-07-02 14:30:00-04:00  220.029999  220.044998  219.820007  219.839996   
2024-07-02 14:31:00-04:00  219.846893  219.854996  219.720001  219.744705   
2024-07-02 14:32:00-04:00  219.764999  219.800003  219.669998  219.729996   
2024-07-02 14:33:00-04:00  219.729996  219.860001  219.680099  219.720001   
...                               ...         ...         ...         ...   
2024-07-05 15:55:00-04:00  225.975006  226.199997  225.919998  226.182297   
2024-07-05 15:56:00-04:00  226.190002  226.250000  226.139999  226.244995   
2024-07-05 15:57:00-04:00  226.242798  226.345001  226.169998  226.315002   
2024-07-05 15:58:00-04:00  226.315002  226.437500  226.264999  226.425003   
2024-07-05 15:59:00-04:00  226.425003  226.449997  226.279999  226.309998   

# 4. Tinkoff API for russion stocks

In [171]:
import os
from datetime import datetime, timedelta
from tinkoff.invest import CandleInterval, Client, InstrumentStatus
from tinkoff.invest.schemas import CandleSource
from tinkoff.invest.utils import now

# Ensure you have the necessary Tinkoff API client library installed and imported correctly

TOKEN = 't.hSZNWYEejBjbpOUElGGm7MbaL9xH5Lccgy3i6BElx9x3N7GSLvZpiIazGQxWJHwddVJ3KAFh0tA8P6wJYZi0WA'  # Replace with your actual Tinkoff API token

# Define instrument ID (FIGI) for GAZP.ME or any other instrument
instrument_id = "BBG004730RP0"  # Replace with the correct FIGI for your instrument

with Client(TOKEN) as client:
    tool = client.instruments
    r = pd.DataFrame(tool.shares(instrument_status=InstrumentStatus.INSTRUMENT_STATUS_ALL).instruments,
    columns=['name', 'figi','ticker','class_code']
                    )
    for candle in client.get_all_candles(
        instrument_id="BBG004730RP0",
        from_=now() - timedelta(days=60),
        interval=CandleInterval.CANDLE_INTERVAL_MONTH,
        candle_source_type=CandleSource.CANDLE_SOURCE_UNSPECIFIED,
    ):
        print(candle)


HistoricCandle(open=Quotation(units=157, nano=630000000), high=Quotation(units=168, nano=150000000), low=Quotation(units=157, nano=430000000), close=Quotation(units=163, nano=220000000), volume=59265880, time=datetime.datetime(2024, 4, 1, 0, 0, tzinfo=datetime.timezone.utc), is_complete=True, candle_source=<CandleSource.CANDLE_SOURCE_EXCHANGE: 1>)
HistoricCandle(open=Quotation(units=163, nano=290000000), high=Quotation(units=165, nano=360000000), low=Quotation(units=123, nano=260000000), close=Quotation(units=126, nano=460000000), volume=124745825, time=datetime.datetime(2024, 5, 1, 0, 0, tzinfo=datetime.timezone.utc), is_complete=True, candle_source=<CandleSource.CANDLE_SOURCE_EXCHANGE: 1>)
HistoricCandle(open=Quotation(units=126, nano=680000000), high=Quotation(units=128, nano=610000000), low=Quotation(units=110, nano=0), close=Quotation(units=116, nano=430000000), volume=92964454, time=datetime.datetime(2024, 6, 1, 0, 0, tzinfo=datetime.timezone.utc), is_complete=False, candle_sourc

In [165]:
r.loc[r.name == 'Газпром']

,name,figi,ticker,class_code
449,Газпром,TCS207661625,RU0007661625,TILA21
902,Газпром,TCSS07661625,GAZP,SPEQ
1176,Газпром,TCS007661625,GAZP,SMAL
8483,Газпром,BBG004730RP0,GAZP,TQBR


In [19]:
import ccxt
import pandas as pd

# Создайте экземпляр биржи Binance
binance = ccxt.binance()

# Загрузите исторические данные по паре BTC/USDT
since = binance.parse8601('2018-02-08T00:00:00Z')
ohlcv = binance.fetch_ohlcv('BTC/USDT', '1h', since, limit=100)

# Преобразуйте данные в DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Выведите полученные данные
df.head(50)

,timestamp,open,high,low,close,volume
0,2018-02-08 00:00:00,7599.00,7844.00,7572.09,7784.02,1521.537318
1,2018-02-09 10:00:00,7789.90,8390.00,7789.90,8269.84,2131.598280
2,2018-02-09 11:00:00,8260.24,8395.00,8227.82,8280.11,1352.206959
3,2018-02-09 12:00:00,8288.03,8300.00,8201.95,8241.00,1003.312642
4,2018-02-09 13:00:00,8241.00,8340.00,8225.00,8309.95,1169.015154
5,2018-02-09 14:00:00,8309.95,8540.00,8285.15,8450.49,2399.571661
6,2018-02-09 15:00:00,8460.97,8510.00,8312.00,8333.91,1471.127073
7,2018-02-09 16:00:00,8333.98,8432.51,8312.00,8383.11,1189.651658
8,2018-02-09 17:00:00,8383.11,8718.00,8372.69,8625.00,2086.817085
9,2018-02-09 18:00:00,8627.00,8738.00,8565.01,8624.60,1615.835729


In [29]:
df = pd.read_csv('../historical_data/btc_history.csv')
df['Timestamp'] = df['Timestamp'].apply(lambda a: datetime.fromtimestamp(a))
df

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [149]:
new_btc_data = btc_data.copy()

In [150]:
new_btc_data['Open time'] = pd.to_datetime(new_btc_data['Open time'])

In [151]:
new_btc_data['Open time'] = pd.to_datetime(new_btc_data['Open time'])
new_btc_data.set_index('Open time', inplace=True)

In [152]:
new_btc_data.head()

,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
Open time,,,,,,,,,,,
2018-02-01 00:00:00,10285.10000000,10335.00000000,10215.07000000,10263.18000000,594.44129000,2018-02-01 00:59:59.999,6115555.42732146,5895,279.12304600,2872194.05048160,0
2018-02-01 01:00:00,10263.18000000,10328.98000000,10216.00000000,10247.49000000,591.51572500,2018-02-01 01:59:59.999,6075265.65422887,7011,206.87583500,2125737.84705670,0
2018-02-01 02:00:00,10249.43000000,10317.73000000,10176.89000000,10199.61000000,479.41256200,2018-02-01 02:59:59.999,4925487.64559189,6550,208.08301600,2138136.03796232,0
2018-02-01 03:00:00,10199.61000000,10250.79000000,9959.04000000,10069.80000000,739.43530900,2018-02-01 03:59:59.999,7459189.67996952,9014,319.40746200,3218742.58923152,0
2018-02-01 04:00:00,10069.77000000,10256.00000000,10000.01000000,10245.79000000,649.93985400,2018-02-01 04:59:59.999,6582218.41977594,7431,387.99948300,3929931.38684970,0


In [158]:
pd.date_range(start='2024-07-02 00:00:00', end='2024-07-20 00:00:00', freq='3d')

DatetimeIndex(['2024-07-02', '2024-07-05', '2024-07-08', '2024-07-11',
               '2024-07-14', '2024-07-17', '2024-07-20'],
              dtype='datetime64[ns]', freq='3D')

In [156]:
full_time_index = pd.date_range(start='2/1/2018', end='2/28/2018 23:00:00', freq='1h')
full_time = pd.DataFrame(full_time_index, columns=['Open time']).set_index('Open time')
full_time[165:218]

""
Open time
2018-02-07 21:00:00
2018-02-07 22:00:00
2018-02-07 23:00:00
2018-02-08 00:00:00
2018-02-08 01:00:00
2018-02-08 02:00:00
2018-02-08 03:00:00
2018-02-08 04:00:00
2018-02-08 05:00:00


In [154]:
new_df = full_time.merge(btc_data, on='Open time', how='outer')

In [157]:
new_df[165:218]

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
165,2018-02-07 21:00:00,8126.96000000,8259.12000000,8000.00000000,8080.04000000,1103.98055600,2018-02-07 21:59:59.999,8965094.03124953,11695.0,500.46729500,4067505.04451966,0
166,2018-02-07 22:00:00,8073.00000000,8078.26000000,7732.35000000,7781.00000000,2984.32312600,2018-02-07 22:59:59.999,23471550.51279152,20148.0,1211.10782100,9527480.37007567,0
167,2018-02-07 23:00:00,7781.00000000,7818.06000000,7531.11000000,7599.00000000,4304.27775200,2018-02-07 23:59:59.999,32850404.38357698,26100.0,1668.70723400,12746735.51575460,0
168,2018-02-08 00:00:00,7599.00000000,7844.00000000,7572.09000000,7784.02000000,1521.53731800,2018-02-08 00:28:14.788,11770168.04386595,12417.0,844.25881300,6532638.63751892,0
169,2018-02-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
170,2018-02-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
171,2018-02-08 03:00:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
172,2018-02-08 04:00:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
173,2018-02-08 05:00:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
174,2018-02-08 06:00:00,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [120]:
for col in new_df.select_dtypes(include='object').columns:
    if col != 'Open time' and col != 'Close time':
        new_df[col] = pd.to_numeric(new_df[col], errors='coerce')

In [141]:
new_df.dtypes

Open                                   float64
High                                   float64
Low                                    float64
Close                                  float64
Volume                                 float64
Close time                      datetime64[ns]
Quote asset volume                     float64
Number of trades                       float64
Taker buy base asset volume            float64
Taker buy quote asset volume           float64
Ignore                                 float64
dtype: object

In [142]:
new_df.index = pd.DatetimeIndex(new_df.index)

In [161]:
pd.Timedelta(months=1, days=0,hours=23, minutes=59, seconds=59, milliseconds=999)

ValueError: cannot construct a Timedelta from the passed arguments, allowed keywords are [weeks, days, hours, minutes, seconds, milliseconds, microseconds, nanoseconds]

In [143]:
new_df.index

DatetimeIndex(['2018-02-01 00:00:00', '2018-02-01 01:00:00',
               '2018-02-01 02:00:00', '2018-02-01 03:00:00',
               '2018-02-01 04:00:00', '2018-02-01 05:00:00',
               '2018-02-01 06:00:00', '2018-02-01 07:00:00',
               '2018-02-01 08:00:00', '2018-02-01 09:00:00',
               ...
               '2018-02-28 14:00:00', '2018-02-28 15:00:00',
               '2018-02-28 16:00:00', '2018-02-28 17:00:00',
               '2018-02-28 18:00:00', '2018-02-28 19:00:00',
               '2018-02-28 20:00:00', '2018-02-28 21:00:00',
               '2018-02-28 22:00:00', '2018-02-28 23:00:00'],
              dtype='datetime64[ns]', name='Open time', length=672, freq=None)

In [144]:


new_df.drop(columns=['Close time']).interpolate(method='time')[165:218]  # 'pad' 'linear'

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
Open time,,,,,,,,,,
2018-02-07 21:00:00,8126.960000,8259.120000,8000.000000,8080.040000,1103.980556,8.965094e+06,11695.000000,500.467295,4.067505e+06,0.0
2018-02-07 22:00:00,8073.000000,8078.260000,7732.350000,7781.000000,2984.323126,2.347155e+07,20148.000000,1211.107821,9.527480e+06,0.0
2018-02-07 23:00:00,7781.000000,7818.060000,7531.110000,7599.000000,4304.277752,3.285040e+07,26100.000000,1668.707234,1.274674e+07,0.0
2018-02-08 00:00:00,7599.000000,7844.000000,7572.090000,7784.020000,1521.537318,1.177017e+07,12417.000000,844.258813,6.532639e+06,0.0
2018-02-08 01:00:00,7604.614706,7860.058824,7578.496176,7798.308824,1539.480287,1.194218e+07,12531.617647,848.319589,6.578948e+06,0.0
2018-02-08 02:00:00,7610.229412,7876.117647,7584.902353,7812.597647,1557.423257,1.211420e+07,12646.235294,852.380365,6.625257e+06,0.0
2018-02-08 03:00:00,7615.844118,7892.176471,7591.308529,7826.886471,1575.366226,1.228622e+07,12760.852941,856.441141,6.671566e+06,0.0
2018-02-08 04:00:00,7621.458824,7908.235294,7597.714706,7841.175294,1593.309196,1.245823e+07,12875.470588,860.501917,6.717875e+06,0.0
2018-02-08 05:00:00,7627.073529,7924.294118,7604.120882,7855.464118,1611.252165,1.263025e+07,12990.088235,864.562693,6.764185e+06,0.0


In [31]:
df.dropna().head(50)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 09:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,2011-12-31 17:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,2011-12-31 18:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,2011-12-31 19:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,2012-01-01 06:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
1896,2012-01-01 17:28:00,4.84,4.84,4.84,4.84,10.000000,48.400000,4.840000
2333,2012-01-02 00:45:00,5.00,5.00,5.00,5.00,10.100000,50.500000,5.000000
3612,2012-01-02 22:04:00,5.00,5.00,5.00,5.00,19.048000,95.240000,5.000000
4553,2012-01-03 13:45:00,5.32,5.32,5.32,5.32,2.419173,12.870000,5.320000
4710,2012-01-03 16:22:00,5.14,5.14,5.14,5.14,0.680000,3.495200,5.140000


In [222]:
test_csv = pd.read_csv('test_csv_missing_fill.csv')

In [223]:
pd.to_datetime(test_csv['Open_time'][0]) + timedelta(hours=1) - timedelta(milliseconds=1)

Timestamp('2018-05-04 08:59:59.999000')

In [224]:
test_csv[1260:1310]

,Open_time,Open,High,Low,Close,Volume,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Currency,Interval,Close_time,Source
1260,2018-06-25 20:00:00,0.476210,0.481230,0.475610,0.478980,9.505695e+05,4.543304e+05,579.000000,6.546835e+05,312871.989834,XRPUSDT,1h,2018-06-25 20:59:59,binance_api
1261,2018-06-25 21:00:00,0.477740,0.481000,0.475570,0.481000,1.289562e+06,6.170606e+05,769.000000,8.381428e+05,401054.184168,XRPUSDT,1h,2018-06-25 21:59:59,binance_api
1262,2018-06-25 22:00:00,0.480030,0.481770,0.477600,0.479370,1.040342e+06,4.995256e+05,603.000000,6.380047e+05,306486.867404,XRPUSDT,1h,2018-06-25 22:59:59,binance_api
1263,2018-06-25 23:00:00,0.479370,0.482450,0.478230,0.481250,1.066048e+06,5.123654e+05,644.000000,7.568703e+05,363837.400675,XRPUSDT,1h,2018-06-25 23:59:59,binance_api
1264,2018-06-26 00:00:00,0.481250,0.482410,0.474010,0.474010,1.075179e+06,5.151238e+05,649.000000,5.937321e+05,284885.002979,XRPUSDT,1h,2018-06-26 00:59:59,binance_api
1265,2018-06-26 01:00:00,0.475130,0.480000,0.474000,0.478790,1.081063e+06,5.157564e+05,838.000000,7.597208e+05,362576.795443,XRPUSDT,1h,2018-06-26 01:59:59,binance_api
1266,2018-06-26 02:00:00,0.475386,0.479925,0.473927,0.478634,1.096367e+06,5.230955e+05,842.363636,7.481167e+05,357061.517596,XRPUSDT,1h,2018-06-26 02:59:59,pandas_interpolate
1267,2018-06-26 03:00:00,0.475643,0.479849,0.473855,0.478477,1.111671e+06,5.304346e+05,846.727273,7.365127e+05,351546.239750,XRPUSDT,1h,2018-06-26 03:59:59,pandas_interpolate
1268,2018-06-26 04:00:00,0.475899,0.479774,0.473782,0.478321,1.126975e+06,5.377737e+05,851.090909,7.249086e+05,346030.961903,XRPUSDT,1h,2018-06-26 04:59:59,pandas_interpolate
1269,2018-06-26 05:00:00,0.476155,0.479698,0.473709,0.478165,1.142279e+06,5.451128e+05,855.454545,7.133045e+05,340515.684057,XRPUSDT,1h,2018-06-26 05:59:59,pandas_interpolate


##### 

In [198]:
test_csv.dtypes

Open_time                        object
Open                            float64
High                            float64
Low                             float64
Close                           float64
Volume                          float64
Quote_asset_volume              float64
Number_of_trades                float64
Taker_buy_base_asset_volume     float64
Taker_buy_quote_asset_volume    float64
Currency                         object
Interval                         object
Close_time                       object
Source                           object
Open_time.1                      object
dtype: object

In [229]:
test = pd.read_csv('test_file.csv')
null_values = test.isnull()

In [264]:
print(*[el.strftime('%Y-%m-%d %H:%M:%S') for el in btc_data_2['Open time']])

2023-03-24 00:00:00 2023-03-24 01:00:00 2023-03-24 02:00:00 2023-03-24 03:00:00 2023-03-24 04:00:00 2023-03-24 05:00:00 2023-03-24 06:00:00 2023-03-24 07:00:00 2023-03-24 08:00:00 2023-03-24 09:00:00 2023-03-24 10:00:00 2023-03-24 11:00:00 2023-03-24 12:00:00 2023-03-24 14:00:00 2023-03-24 15:00:00 2023-03-24 16:00:00 2023-03-24 17:00:00 2023-03-24 18:00:00 2023-03-24 19:00:00 2023-03-24 20:00:00 2023-03-24 21:00:00 2023-03-24 22:00:00 2023-03-24 23:00:00


In [265]:
[1, 2, 3, 4][-6:]

[1, 2, 3, 4]

In [10]:
params = {'company_id': 1}
params

{'company_id': 1}

In [12]:
params.keys()

TypeError: 'dict_keys' object is not subscriptable

In [13]:
key, value = next(iter(params.items()))

In [14]:
key, value

('company_id', 1)